In [1]:
# 환경 변수에서 API 키 가져오기
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

In [2]:
# CrewAI 라이브러리에서 필요한 클래스 가져오기
from langchain_openai import ChatOpenAI

# LLM
llm = ChatOpenAI(
    #model='gpt-3.5-turbo', 
    model='gpt-4o-mini',
    temperature=0,
    api_key=OPENAI_API_KEY)

In [2]:
# Search Tool
from crewai import Agent, Task, Crew, Process
from crewai.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults


#search_tool = TavilySearchResults(api_key=TAVILY_API_KEY)
@tool("tavily_search_wrapper")
def tavily_search_wrapper(query: str) -> str:
    """Tavily 검색 도구 래퍼"""
    search_tool = TavilySearchResults(api_key=TAVILY_API_KEY)
    return search_tool.run(query)

ModuleNotFoundError: No module named 'crewai'

In [4]:
# Agent
researcher = Agent(
role='Market Researcher',
goal='Uncover emerging trends and investment opportunities in the cryptocurrency market in 2024.',
backstory='Identify groundbreaking trends and actionable insights.',
verbose=True,
tools=[tavily_search_wrapper],
allow_delegation=False,
llm=llm,
max_iter=3,
max_rpm=10,
)

analyst = Agent(
role='Investment Analyst',
goal='Analyze cryptocurrency market data to extract actionable insights and investment leads.',
backstory='Draw meaningful conclusions from cryptocurrency market data.',
verbose=True,
allow_delegation=False,
llm=llm,
)

In [5]:
# Tasks
research_task = Task(
description='Explore the internet to pinpoint emerging trends and potential investment opportunities.',
agent=researcher,
expected_output='A detailed summary of the reserch results in string format'
)


analyst_task = Task(
description='Analyze the provided cryptocurrency market data to extract key insights and compile a concise report.',
agent=analyst,
expected_output='A refined finalized version of the report in string format'
)

In [6]:
#`Crew` is a group of agents working together to accomplish a task
crypto_crew = Crew(
agents=[researcher, analyst],
tasks=[research_task, analyst_task],
process=Process.sequential  
)

In [7]:
#`kickoff` method starts the crew's process
result = crypto_crew.kickoff()

# Agent: Market Researcher
## Task: Explore the internet to pinpoint emerging trends and potential investment opportunities.


# Agent: Market Researcher
## Thought: I need to explore the internet for emerging trends and investment opportunities in the cryptocurrency market for 2024.
## Using tool: tavily_search_wrapper
## Tool Input: 
"{\"query\": \"emerging trends and investment opportunities in cryptocurrency market 2024\"}"
## Tool Output: 
[{'title': 'Cryptocurrency Market Size, Growth & Trends Report, 2024', 'url': 'https://planwriters.com/blog/cryptocurrency-market-size-growth-trends-report-2024', 'content': 'With a projected market size of $2 trillion and emerging trends like DeFi, NFTs, and green cryptocurrencies, the industry is ripe for', 'score': 0.9158166}, {'title': 'The Cryptocurrency Industry in 2024: A Year in Review - Nonco', 'url': 'https://nonco.com/the-cryptocurrency-industry-in-2024-a-year-in-review/', 'content': 'The total cryptocurrency market capitalization gre